preprocessing
vectorization - converting text to numeric
    tfidf
    Word2vec
    glove
    Bert (https://www.kdnuggets.com/2022/01/finetuning-bert-tweets-classification-ft-hugging-face.html)
    Fastext
    Elmo
    XLNet
    Transformers (https://pub.towardsai.net/text-classification-using-transformers-a2c6b3395ce3)
Reduce the dimensions
    Principal Component Analysis
    Singular Value Decomposition
    Latent Semantic Indexing
    Pooling
    multi-dimensional scaling
calculate imbalanced ratio and find an oversampling ratio
oversample the imbalanced dataset using unlabeled data
    define a similarity function
(extension) define a strategy to choose unlabeled instances wisely to reduce computation time
    clustering?
    LDA
    Non-negative Matrix Factorization
    K-NN based graph approach (2 paper’s approaches)
it should be trained to maximize the within-class similarity while minimizing between-class similarity using labeled data using the labeled data
using the defined similarity function calculate similarity for each unlabeled instance of each class
define a similarity function
similarity functions for vectors
    Measurement of Text Similarity: A Survey: a very detailed survey of similarity functions that are used for text data
    cosine similarity
    minkowski family (euclidean, manhattan)
    hamming distance
    Jaccard index
    Sorensen-dice index
    KL divergence
    Jensen–Shannon divergence with LDA
    Wasserstein distance
    SMTP 
    word mover’s distance
to each labeled instance (possible to create a confidence)
take the average of the similarity
take the min max average and std and decide using these
to each class centroids (generalization)
use all instances
use only the instances that are similar to each other to ensure data confidence (excluding outliers)
if the similarity exceeds some defined threshold assign them to related classes
train a classifier and look for an improvement

clustering approach
does one vs rest classifier work?
if it works use self-supervised methods
Train a final classifier with oversampled data

Experimentation with different datasets
ANOVA for parameters to select important parameters
Tune important parameters to enhance performance
Compare the two cases to show the improvement
Compare with different solutions?


In [1]:
import pandas as pd
import numpy as np
import preprocess
import utilities
import seaborn as sns
from numpy import dot
from numpy.linalg import norm
from itertools import combinations
import matplotlib.pyplot as plt

scenarios:
    - preprocessing
        var
        yok
        different preprocessing
            only remove noise
            do not lower
    - vectorizers
    - similarity functions

In [2]:
data_paths = {'opp115'   : r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv',
              'ohsumed'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\ohsumed.csv',
              'reuters'  : r'C:\Users\IsmailKaraman\workspace\GitHub\thesis\data\Reuters21578.csv'}

In [3]:
sentence_embeddings =  ['all-MiniLM-L6-v2',
                        'all-MiniLM-L12-v2',
                        'all-mpnet-base-v2',
                        'all-distilroberta-v1',
                        'bert-base-nli-mean-tokens',
                        'distiluse-base-multilingual-cased-v1',
                        'multi-qa-mpnet-base-dot-v1',
                        'paraphrase-MiniLM-L6-v2',
                        'paraphrase-multilingual-MiniLM-L12-v2',
                        'paraphrase-mpnet-base-v2',
                        'paraphrase-xlm-r-multilingual-v1',
                        'sentence-t5-large',
                        'stsb-roberta-large']

In [4]:
df = utilities.read_data(data_paths['opp115'])

In [5]:
X = df['text']
y = df.drop(['text'], axis=1)

In [6]:
def calculate_between_class_similarities(col1, col2, X, y):
    
    similarities = []
    
    for idx1 in y[y[col1]==1].index:
        for idx2 in y[y[col2]==1].index:
            similarities.append(similarities.vector_similarity(X.loc[idx1], X.loc[idx2]))
    
    return sum(similarities)/len(similarities)     

In [7]:
def calculate_similarity_matrix(X, y, sim_method='cosine'):
    
    import similarities
    
    sim_df = pd.DataFrame(index=y.columns, columns=y.columns)
    
    for col in y.columns:
    
        indexes = y[y[col]==1].index
        print(col)
        print(X.loc[indexes].shape)
        sim_df.loc[col, col] = similarities.calculate_within_class_similarity(X.loc[indexes])
    
    for col1, col2 in list(combinations(y.columns, 2)):
        sim_df.loc[col1, col2] = calculate_between_class_similarities(col1, col2, X, y)
    
    return sim_df

In [8]:
for embedding_method in sentence_embeddings:
    
    X_num = utilities.vectorize_data(X, embedding_method)
    X_num = pd.Series([np.squeeze(i) for i in X_num])
    sim_df = calculate_similarity_matrix(X_num, y)

    plt.figure()
    sns.heatmap(sim_df.fillna(0), annot=True,
    xticklabels=sim_df.columns,
    yticklabels=sim_df.columns, cmap="BuPu")

    

Data Retention
(78,)
Data Security
(207,)
Do Not Track
(31,)
First Party Collection/Use
(1181,)
International and Specific Audiences
(296,)
Introductory/Generic
(378,)
Policy Change
(116,)
Practice not covered
(129,)
Privacy contact information
(202,)
Third Party Sharing/Collection
(931,)
User Access, Edit and Deletion
(147,)
User Choice/Control
(352,)


AttributeError: 'list' object has no attribute 'vector_similarity'

In [ ]:
stop

In [ ]:
def cosine_similarity(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0:
        norm1 += 0.00001
    if norm2 == 0:
        norm2 += 0.00001   
    return np.dot(vec1, vec2)/(norm1*norm2)

In [ ]:
cosine_similarity(X_num[0], X_num[0])

In [ ]:

model = AutoModel.from_pretrained(\"nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("saibo/legal-roberta-base")
                                  
tokenizer = AutoTokenizer.from_pretrained('saibo/legal-roberta-base')   

In [ ]:
adasd

save sim_df adnd run again to compare results

In [ ]:
sentence1 = 'I really love to play football'
sentence2 = 'Playing football is my passion.'

In [ ]:
def transform_sentence(sentence1, sentence2, model, preprocessing=False):
    
    model = SentenceTransformer(model)
    
    if preprocessing:
        import preprocess
        sentence1 = preprocess.preprocess_text(sentence1)
        sentence2 = preprocess.preprocess_text(sentence2)
        
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)
    
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    
    return cosine_scores

In [ ]:

# multi-qa-mpnet-base-dot-v1 - https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-dot-v1



In [ ]:
from transformers import AutoTokenizer, AutoModel

import torch


#Mean Pooling - Take attention mask into account for correct averaging

def mean_pooling(model_output, attention_mask):

    token_embeddings = model_output[0] #First element of model_output contains all token embeddings

    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
model_name = 
'saibo/legal-roberta-base'
'nlpaueb/legal-bert-base-uncased'
'nlpaueb/legal-bert-small-uncased'
'saibo/legal-roberta-base'
'albert-base-v2'
'ALBERT-xlarge'
'ALBERT-xxlarg'

tokenizer = AutoTokenizer.from_pretrained(model_name)    
text_tensor1 = tokenizer.encode(sentence1, padding=True, truncation=True, return_tensors='pt')
text_tensor1 = tokenizer.encode(sentence2, padding=True, truncation=True, return_tensors='pt')

model = AutoModel.from_pretrained(model_name)

with torch.no_grad():

    output1 = model(text_tensor1)
    output2 = model(text_tensor2)

sentence_embeddings1 = mean_pooling(output1, text_tensor1)
sentence_embeddings2 = mean_pooling(output2, text_tensor2)

cosine_scores = util.pytorch_cos_sim(sentence_embeddings1, sentence_embeddings2)

print(cosine_scores.item())

print(sentence_embeddings1.shape, sentence_embeddings2.shape)


In [ ]:
model = SentenceTransformer('stsb-roberta-large')

In [ ]:
sentence1 = 'I love to play football because I am a player'
sentence2 = 'Playing football is my passion.'

In [ ]:
embedding1 = model.encode(sentence1, convert_to_tensor=True)
embedding2 = model.encode(sentence2, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print(cosine_scores.item())

embedding1 = model.encode(preprocess.preprocess_text(sentence1), convert_to_tensor=True)
embedding2 = model.encode(preprocess.preprocess_text(sentence2), convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
print(cosine_scores.item())

In [ ]:
https://medium.com/nlplanet/two-minutes-nlp-11-word-embeddings-models-you-should-know-a0581763b9a9

In [ ]:
"http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000400647"

In [ ]:
from re import sub
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

query_string = 'fruit and vegetables'
documents = ['cars drive on the road', 'tomatoes are actually fruit']
stopwords = ['the', 'and', 'are', 'a']

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)',  image_token  doc)
    doc = sub(r'<[^<>]+(>|$)',   doc)
    doc = sub(r'\[img_assist[^]]*?\]',   doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',  url_token  doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float(inf)) if token not in stopwords]

query_string = 'fruit and vegetables'
documents = ['cars drive on the road', 'tomatoes are actually fruit']
documents = ['I like Python because I can build AI applications',
             'I like Python because I can do data analytics'
             'The cat sits on the ground',
             'The cat walks on the sidewalk']

query_string = 'I like Javascript because I can build web applications'
# Preprocess the documents, including the query string

corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)
# Load the model: this is a big file, can take a while to download and open
glove = api.load(glove-wiki-gigaword-50)    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)
                # Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]
index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],

            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents

sorted_indexes = np.argsort(doc_similarity_scores)[::-1]

for idx in sorted_indexes:

    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


# context based models
source: https://medium.com/nlplanet/two-minutes-nlp-11-word-embeddings-models-you-should-know-a0581763b9a9

In [ ]:
elmo
cove
xlm

### fine-tuning sentence-embedding models

it is possible to fine-tune sentence similarity models by using cosine similarity loss function. 
some pair of sentences and similarity between them are given as training data and models can be fine-tuned
by using in-class similarity as high score and between-class similarity as low score we can tune the model to find more instances by the similarity

https://www.sbert.net/docs/training/overview.html

In [ ]:
from re import sub
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

In [ ]:
query_string = 'fruit and vegetables'
documents = ['cars drive on the road', 'tomatoes are actually fruit']

stopwords = ['the', 'and', 'are', 'a']

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

In [ ]:
query_string = 'fruit and vegetables'
documents = ['cars drive on the road', 'tomatoes are actually fruit']

In [ ]:
documents = ["I like Python because I can build AI applications",
          "I like Python because I can do data analytics",
          "The cat sits on the ground",
         "The cat walks on the sidewalk"]

query_string = "I like Javascript because I can build web applications"

In [ ]:
# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)

In [ ]:
# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

In [ ]:
# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


In [ ]:
http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000400647